# Data Visualization

- px.histogram (nbins)

- import plotly.express as px VS matplotlib (pandas), seaborn, ggplot (R but Python), shiny, bokeh, The Pudding (D3.js), https://informationisbeautiful.net ("Data Journalism")
- histograms and barplots and alternatives
- kernel density estimators with seaborn

- location (mean), scale (variance), range, quartiles, skew, modality, outliers...
    - How boxplots define outliers

- Populations / distributions versus samples 

```python 
from scipy import stats
stats.norm(loc=mu, scale=std)
.rvs() and .pdf()
import numpy as np and np.linspace()
px.line
```


# deferred: parameters versus statistics

In [83]:
import pandas as pd
url = "https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-08-11/avatar.csv"
avatar = pd.read_csv(url) #avatar.isnull().sum() #avatar[avatar.isnull().sum(axis=1)>0]
avatar[:10]

,id,book,book_num,chapter,chapter_num,character,full_text,character_words,writer,director,imdb_rating
0,1,Water,1,The Boy in the Iceberg,1,Katara,Water. Earth. Fire. Air. My grandmother used t...,Water. Earth. Fire. Air. My grandmother used t...,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
1,2,Water,1,The Boy in the Iceberg,1,Scene Description,"As the title card fades, the scene opens onto ...",NaN,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
2,3,Water,1,The Boy in the Iceberg,1,Sokka,It's not getting away from me this time. [Clos...,It's not getting away from me this time. Watc...,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
3,4,Water,1,The Boy in the Iceberg,1,Scene Description,"The shot pans quickly from the boy to Katara, ...",NaN,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
4,5,Water,1,The Boy in the Iceberg,1,Katara,"[Happily surprised.] Sokka, look!","Sokka, look!","‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
5,6,Water,1,The Boy in the Iceberg,1,Sokka,"[Close-up of Sokka; whispering.] Sshh! Katara,...","Sshh! Katara, you're going to scare it away. ...","‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
6,7,Water,1,The Boy in the Iceberg,1,Scene Description,"Behind Sokka, Katara is still making circular ...",NaN,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
7,8,Water,1,The Boy in the Iceberg,1,Katara,[Struggling with the water that passes right i...,"But, Sokka! I caught one!","‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
8,9,Water,1,The Boy in the Iceberg,1,Scene Description,The bubble containing her fish slowly drifts a...,NaN,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1
9,10,Water,1,The Boy in the Iceberg,1,Katara,[Exclaims indignantly.] Hey!,Hey!,"‎Michael Dante DiMartino, Bryan Konietzko, Aar...",Dave Filoni,8.1


In [437]:
avatar.dtypes

id                   int64
book                object
book_num             int64
chapter             object
chapter_num          int64
character           object
full_text           object
character_words     object
writer              object
director            object
imdb_rating        float64
dtype: object

In [435]:
avatar.character.value_counts()#[:10]

character
Scene Description    3393
Aang                 1796
Sokka                1639
Katara               1437
Zuko                  776
                     ... 
The Hippo               1
Audience                1
Young Mai               1
Old woman               1
Katara and Sokka        1
Name: count, Length: 374, dtype: int64